In [1]:
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import pandas
import numpy as np
import statistics

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = pandas.read_csv("data/table2.csv", sep="\t")
dataset.head()

,relevant,depth,number_bold,number_br,number_div,number_img,number_li,number_links,number_p,number_relevants,number_td,number_th,number_tr,relevants_ratio
0,0,7,0,0,14,6,0,20,0,80,86,19,16,0.48
1,0,3,1,0,0,0,0,0,0,2,2,1,2,0.33
2,0,5,15,60,0,0,0,0,0,142,5,0,1,0.42
3,1,3,0,0,0,0,0,0,0,60,54,0,27,0.65
4,1,4,7,0,0,0,0,0,0,46,53,53,53,0.30


## analisi features
Addestrando una rete (epochs:5, neurons:16, batch_size:16, activation:'tanh')
    
con KFold(8), utilizzando **una** sola feature si è ottenuto

       1. depth                loss: 0.338, acc: 0.068 (std: 0.137)
       2. number_links         loss: 0.293, acc: 0.437 (std: 0.350)
       3. number_relevants     loss: 0.303, acc: 0.240 (std: 0.216)
       4. number_td            loss: 0.289, acc: 0.440 (std: 0.262)
       5. number_th            loss: 0.302, acc: 0.292 (std: 0.305)
       6. number_tr            loss: 0.287, acc: 0.450 (std: 0.254)

In [8]:
# Dataset to numpy arrays

# choose the target feature and the features to train on
TARGET_FEATURE = 'relevant'
CHOSEN_FEATURES = ['number_img', 'number_td', 'number_tr', 'number_relevants', 'number_links', 'number_bold', 'relevants_ratio']
#CHOSEN_FEATURES = ["relevants_ratio"]

CHOSEN_FEATURES = sorted(CHOSEN_FEATURES) # XXX as convention we sort the features based on their names
print(CHOSEN_FEATURES)

if (TARGET_FEATURE in CHOSEN_FEATURES): CHOSEN_FEATURES.remove(TARGET_FEATURE) # ensure we do not use target feature

data  = np.ndarray((len(dataset), len(CHOSEN_FEATURES)), np.float32)
label = np.ndarray((len(dataset), 2), np.float32)

for i, row in dataset.iterrows():
    label[i] = np.zeros(2)
    label[i][int(row[TARGET_FEATURE])] = 1

    data[i]  = np.fromiter([row[feature] for feature in CHOSEN_FEATURES], np.float32)
    
print(data[0])

['number_bold', 'number_img', 'number_links', 'number_relevants', 'number_td', 'number_tr', 'relevants_ratio']
[ 0.    6.   20.   80.   86.   16.    0.48]


In [13]:
# model definition

def table_classifier(neuron, activation, input_shape=()):
    model = Sequential()
    model.add(Dense(neuron, input_shape=input_shape, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer="adam", 
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [24]:
# train & test
import itertools

loo = KFold(6)

epochs = [30]
batch_size = [8]
neurons = [32, 64]
activation = ['tanh', 'relu', 'relu']

hyperparams = list(itertools.product(epochs, batch_size, neurons, activation))
hyperparams

[(30, 8, 32, 'tanh'),
 (30, 8, 32, 'relu'),
 (30, 8, 32, 'relu'),
 (30, 8, 64, 'tanh'),
 (30, 8, 64, 'relu'),
 (30, 8, 64, 'relu')]

In [25]:
early_stop_val_acc = EarlyStopping(monitor = "val_loss", min_delta=0.005, patience=5, verbose=True)

for epoch, batch, neuron, activation in hyperparams:

    res = []
    for (i, (train_index, test_index)) in enumerate(loo.split(data)):
        
        model = table_classifier(neuron, activation, input_shape=(len(CHOSEN_FEATURES),))
        
        data_train,   data_test =  data[train_index],  data[test_index]
        label_train, label_test = label[train_index], label[test_index]
        
        
        t = model.fit(data_train, label_train, epochs=epoch, batch_size=batch, shuffle=True, verbose=False,
                 validation_split=0.3, callbacks=[early_stop_val_acc])
        
        r = model.evaluate(data_test, label_test, verbose=False)
        print(r)
        res.append(r)
        
    loss = statistics.mean(list(map(lambda x: x[0], res)))
    acc  = statistics.mean(list(map(lambda x: x[1], res)))
    acc_std = statistics.stdev(list(map(lambda x: x[1], res)))
    
    print("({}, {}, {}, {})  - loss: {}, acc: {} (std: {})".format(epoch, batch, neuron, activation,
                                                                   loss, acc, acc_std))
    print()

Epoch 00026: early stopping
[0.10324679405474271, 0.9704918032786886]
Epoch 00025: early stopping
[0.1695456682047883, 0.9344262295081968]
Epoch 00026: early stopping
[0.13122763430974524, 0.9573770491803278]
Epoch 00026: early stopping
[0.16870917440926442, 0.9475409836065574]
Epoch 00021: early stopping
[0.10374179953023006, 0.9572368421052632]
Epoch 00027: early stopping
[0.11156018686137702, 0.9539473684210527]
(30, 8, 32, tanh)  - loss: 0.1313385428950246, acc: 0.9535033793500144 (std: 0.011976940720095577)

Epoch 00018: early stopping
[0.14467889730070457, 0.9245901645207014]
Epoch 00021: early stopping
[0.17506640809969823, 0.9311475411790316]
Epoch 00027: early stopping
[0.14218943831739855, 0.9180327868852459]
Epoch 00023: early stopping
[0.1795263596978344, 0.9344262297036218]
Epoch 00015: early stopping
[0.1285712193501623, 0.9407894736842105]
Epoch 00018: early stopping
[0.13427417078300527, 0.9342105263157895]
(30, 8, 32, relu)  - loss: 0.15071774892480055, acc: 0.93053278

In [26]:
## set the best hyperparameters from the results above
best_epochs = 30    
best_batch_size = 8
best_neurons = 32
best_activation = 'tanh'

In [27]:
## retrain the model on the whole dataset and save it
model = table_classifier(best_neurons, best_activation, input_shape=(len(CHOSEN_FEATURES),))

model.fit(data, label, epochs=best_epochs, 
                       batch_size=best_epochs, 
                       shuffle=True, verbose=True)

model.save("models/table_classifier.h5")

Epoch 1/30
1828/1828 [==============================] - 2s 1ms/step - loss: 1.0709 - acc: 0.3468
Epoch 2/30
1828/1828 [==============================] - 0s 77us/step - loss: 0.5372 - acc: 0.7117
Epoch 3/30
1828/1828 [==============================] - 0s 77us/step - loss: 0.3324 - acc: 0.8911
Epoch 4/30
1828/1828 [==============================] - 0s 119us/step - loss: 0.2686 - acc: 0.9141
Epoch 5/30
1828/1828 [==============================] - 0s 90us/step - loss: 0.2322 - acc: 0.9272
Epoch 6/30
1828/1828 [==============================] - 0s 60us/step - loss: 0.2079 - acc: 0.9256
Epoch 7/30
1828/1828 [==============================] - 0s 61us/step - loss: 0.1905 - acc: 0.9272
Epoch 8/30
1828/1828 [==============================] - 0s 61us/step - loss: 0.1790 - acc: 0.9338
Epoch 9/30
1828/1828 [==============================] - 0s 60us/step - loss: 0.1710 - acc: 0.9327
Epoch 10/30
1828/1828 [==============================] - 0s 57us/step - loss: 0.1655 - acc: 0.9409
Epoch 11/30
1828/18